## Langchain Output Parsing

In [3]:
import os,sys,getpass
import pprint
os.environ["OPENAI_API_KEY"] = getpass.getpass(prompt='OpenAI API Token:')

In [4]:
from langchain_openai import OpenAI,ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

- Basic LLM with system and user inputs 

In [22]:
llm = ChatOpenAI(model = 'gpt-3.5-turbo')

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a IMF Economist. You job is to respond to user's question from a Macro economic perspective. Follow instructions as closely as you can."),
    ("user", "{input}")
])

chain = prompt | llm 

In [23]:
res = chain.invoke({"input": "Please Provide 5 reasons for stock market volatility. and provide an example for each of them."})
print(res.content)

Stock market volatility can be influenced by various factors. Here are five reasons for stock market volatility along with examples for each:

1. Economic Uncertainty: Uncertainty about economic conditions, such as GDP growth, interest rates, or inflation, can lead to stock market volatility. For example, during the global financial crisis in 2008, uncertainty about the extent of the crisis and its impact on the economy caused stock markets around the world to experience significant volatility.

2. Political Instability: Political events and policy changes can create uncertainty and affect investor sentiment, leading to stock market volatility. For instance, Brexit negotiations and their potential impact on trade relations between the United Kingdom and the European Union have caused volatility in the UK stock market.

3. Corporate Earnings: Company-specific factors, such as earnings reports or unexpected changes in business prospects, can cause stock market volatility. For example, if

- ### Basic List output parser

In [46]:
Prompt = """
Please Provide 5 reasons for stock market volatility. and provide an example for each of them.
Please format it as a numbered list for clarity and ease of understanding. 
For example, structure your answer with each item listed as: 
1) [item 1]
2) [item 2]
and so on.

Please provide your answer below:
"""
res = chain.invoke({"input": Prompt})
print(res.content)

1) Economic indicators: Economic indicators such as GDP growth, inflation, and interest rates can have a significant impact on stock market volatility. For example, if GDP growth slows down, it may indicate a weaker economy, causing investors to sell stocks and leading to increased volatility.

2) Geopolitical events: Geopolitical events such as wars, political instability, or trade disputes can create uncertainty in the financial markets, leading to increased stock market volatility. For instance, when the United States and China engaged in a trade war, it caused significant fluctuations in stock prices.

3) Investor sentiment: Investor sentiment plays a crucial role in stock market volatility. If investors become overly optimistic or pessimistic about the future prospects of the market, it can lead to increased volatility. For example, during the dot-com bubble in the late 1990s, investors had an extremely optimistic outlook, causing stock prices to skyrocket and subsequently crash.


In [119]:
import re 

def parse_list_string(res_str):
    if not isinstance(res_str, str):
        raise ValueError("Input must be a string")
    # Compile the regular expression for efficiency if used multiple times
    pattern = r"\d+\)(.*?)(?=\n|$)"
    # Find all matches and strip whitespace from each item
    items = [item.strip() for item in re.findall(pattern, res_str,re.DOTALL)]
    return items 
    return items 

    

In [124]:
items = parse_list_string('Add additional test as noise to see robustness.\n\n' + res.content + '\n\nAdd additional test as noise to see robustness.')
items

['Economic indicators: Economic indicators such as GDP growth, inflation, and interest rates can have a significant impact on stock market volatility. For example, if GDP growth slows down, it may indicate a weaker economy, causing investors to sell stocks and leading to increased volatility.',
 'Geopolitical events: Geopolitical events such as wars, political instability, or trade disputes can create uncertainty in the financial markets, leading to increased stock market volatility. For instance, when the United States and China engaged in a trade war, it caused significant fluctuations in stock prices.',
 'Investor sentiment: Investor sentiment plays a crucial role in stock market volatility. If investors become overly optimistic or pessimistic about the future prospects of the market, it can lead to increased volatility. For example, during the dot-com bubble in the late 1990s, investors had an extremely optimistic outlook, causing stock prices to skyrocket and subsequently crash.',

- ### output reformat parser
    - it kinda works, but it did change the actual contents a little bit 
    - **not very ideal. when format to json, the first value tends to be very short**

In [131]:
Reformat_Prompt = """
Here is a answer provided by a language model:
-------------
{}
-------------

In the provided output, each item contains a reason with brief description and an example
Please reformat the initial result into a JSON object with the following format: 
{{"results": 
    [ 
        {{"reason": "<reason_description1>", "example": "<example_description1>"}}, 
        {{"reason": "<reason_description2>", "example": "<example_description2>"}}, 
        ...... 
    ]
}}

Please provide your answer below:
"""
print(Reformat_Prompt.format(res.content))


Here is a answer provided by a language model:
-------------
1) Economic indicators: Economic indicators such as GDP growth, inflation, and interest rates can have a significant impact on stock market volatility. For example, if GDP growth slows down, it may indicate a weaker economy, causing investors to sell stocks and leading to increased volatility.

2) Geopolitical events: Geopolitical events such as wars, political instability, or trade disputes can create uncertainty in the financial markets, leading to increased stock market volatility. For instance, when the United States and China engaged in a trade war, it caused significant fluctuations in stock prices.

3) Investor sentiment: Investor sentiment plays a crucial role in stock market volatility. If investors become overly optimistic or pessimistic about the future prospects of the market, it can lead to increased volatility. For example, during the dot-com bubble in the late 1990s, investors had an extremely optimistic outlo

In [133]:
import json
res_json = chain.invoke({"input": Reformat_Prompt.format(res.content)})
print(res_json.content)

{"results": [
    {"reason": "Economic indicators", "example": "If GDP growth slows down, it may indicate a weaker economy, causing investors to sell stocks and leading to increased volatility."},
    {"reason": "Geopolitical events", "example": "When the United States and China engaged in a trade war, it caused significant fluctuations in stock prices."},
    {"reason": "Investor sentiment", "example": "During the dot-com bubble in the late 1990s, investors had an extremely optimistic outlook, causing stock prices to skyrocket and subsequently crash."},
    {"reason": "Corporate earnings announcements", "example": "If a company reports better-than-expected earnings, it can lead to a surge in stock prices."},
    {"reason": "Financial market regulations", "example": "The introduction of the Volcker Rule in the aftermath of the 2008 financial crisis is an example of regulatory changes impacting stock market volatility."}
]}
